#  Process large Glossary

In [1]:
!pwd

/Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks


In [3]:
PROC_DATA_PREFIX = '/Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data'

In [20]:
# Look at data
!wc -l {PROC_DATA_PREFIX}/lexicon40
!head -3 {PROC_DATA_PREFIX}/lexicon40
!tail -3 {PROC_DATA_PREFIX}/lexicon40

  381171 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lexicon40
kesia 	 мбе
antoine 	 антуан
mindua 	 миндуа
participation 	 россии
partner 	 партнера
russia 	 россии


In [21]:
# Sort and remove duplicate lines
!sort -u {PROC_DATA_PREFIX}/lexicon40 > {PROC_DATA_PREFIX}/lexicon_sort_uniq40

In [22]:
!wc -l {PROC_DATA_PREFIX}/lexicon_sort_uniq40

   58241 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lexicon_sort_uniq40


In [23]:
# Remove lines where SRC == TRG
with open(PROC_DATA_PREFIX + '/' + 'lexicon_sort_uniq40', 'r', encoding='utf8') as inF, \
     open(PROC_DATA_PREFIX + '/' + 'lexicon_sort_uniq_nonequal40', 'w', encoding='utf8') as toF:
    for line in inF:
        line_list = line.split('\t')
        src_str, trg_str = line_list[0].strip(), line_list[1].strip()
        if src_str != trg_str:
            toF.write(line)

In [24]:
!wc -l {PROC_DATA_PREFIX}/lexicon_sort_uniq_nonequal40

   58161 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lexicon_sort_uniq_nonequal40


In [25]:
# Remove lines where SRC ~= TRG
import nltk
from nltk.metrics.distance import jaro_similarity

# Count all similarity scores
with open(PROC_DATA_PREFIX + '/' + 'lexicon_sort_uniq_nonequal40', 'r', encoding='utf8') as inF, \
    open(PROC_DATA_PREFIX + '/' + 'lex_sim_scores.txt', 'w', encoding='utf8') as toF_fuzzy:
    for line in inF:
        line_list = line.split('\t')
        src_str, trg_str = line_list[0].strip(), line_list[1].strip()
        jaro_sim = jaro_similarity(src_str, trg_str)
        toF_fuzzy.write(str(jaro_sim) + '\n')

In [26]:
!wc -l {PROC_DATA_PREFIX}/'lex_sim_scores.txt'

   58161 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lex_sim_scores.txt


In [27]:
# Look at highly similar items 
with open(PROC_DATA_PREFIX + '/' + 'lex_sim_scores.txt', 'r', encoding='utf8') as inF_scores, \
    open(PROC_DATA_PREFIX + '/' + 'lexicon_sort_uniq_nonequal40', 'r', encoding='utf8') as inF_text, \
    open(PROC_DATA_PREFIX + '/' + 'lex_above_45_sim40.txt', 'w', encoding='utf8') as toF_text:
    simil_scores = [float(s) for s in inF_scores]
    print(len(simil_scores))
    for i, line in enumerate(inF_text):
        if simil_scores[i] > 0.45:
            toF_text.write(line)

!wc -l {PROC_DATA_PREFIX}/'lex_above_45_sim40.txt'

58161
     832 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lex_above_45_sim40.txt


In [28]:
# This is what I am going to remove from Flossary
!head {PROC_DATA_PREFIX}/'lex_above_45_sim40.txt'
!tail {PROC_DATA_PREFIX}/'lex_above_45_sim40.txt'

10th 	 10го
131st 	 131м
17th 	 17м
18th 	 18й
1980s 	 1980х
1b 	 1в
252nd 	 252го
2nd 	 2я
2x9 	 2х9
319th 	 319м
required 	 мнсi
res 	 r
sensitivity 	 gsinas
set 	 rev
statement 	 prst
statute 	 ter
terms 	 mutandis
tribunal 	 ter
use 	 подтвeрждая
with 	 bis


In [29]:
# Keep only dissimilar items
threshold = 0.45  # similarity score
with open(PROC_DATA_PREFIX + '/' + 'lex_sim_scores.txt', 'r', encoding='utf8') as inF_scores, \
    open(PROC_DATA_PREFIX + '/' + 'lexicon_sort_uniq_nonequal40', 'r', encoding='utf8') as inF_text, \
    open(PROC_DATA_PREFIX + '/' + 'lex_preproc40', 'w', encoding='utf8') as toF_text:
    simil_scores = [float(s) for s in inF_scores]
    print(len(simil_scores))
    for i, line in enumerate(inF_text):
        if simil_scores[i] < threshold:
            toF_text.write(line)

!wc -l {PROC_DATA_PREFIX}/'lex_preproc40'

58161
   57329 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lex_preproc40


In [30]:
# remove more extraneous elements, digits
stopwords = ['--']
digits_set = set('0123456789')
with open(PROC_DATA_PREFIX + '/' + 'lex_preproc40', 'r', encoding='utf8') as inF, \
    open(PROC_DATA_PREFIX + '/' + 'lex_preproc40_cleaned', 'w', encoding='utf8') as toF:
    for line in inF:
        line_list = line.split('\t')
        src_str, trg_str = line_list[0].strip(), line_list[1].strip()
        if  (len(src_str) > 1) \
            and (len(src_str) > 1) \
            and (len(set(src_str) & digits_set) == 0) \
            and (len(set(trg_str) & digits_set) == 0) \
            and (trg_str not in stopwords):
            toF.write(line)

!wc -l {PROC_DATA_PREFIX}/'lex_preproc40_cleaned'

   56460 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lex_preproc40_cleaned
